# **ATIVIDADE 4 - MLP**

## **1.Introdução**

### 1.1.MLP - Multilayer Perceptron

Essa atividade devemos utilizar o MLP (Multilayer Percepetron). Que é um tipo de rede neural artificial composta por múltiplas camadas de neurônios, incluindo uma camada de entrada, uma ou mais camadas ocultas, e uma camada de saída. Cada neurônio utiliza uma função de ativação para processar os dados e aprender padrões complexos.

Podemos entender o MLP como um **aproximador universal de funções**. Pois, com uma camada oculta suficientemente grande e a função de ativação certa, ela pode aproximar qualquer função contínua com precisão.

### 1.2.Método

Nesta tarefa, iremos utilizar as mesmas bases que utilizamos nos métodos anteriores e iremos fazer da seguinte forma:

* Pegar a base com maior número de colunas, pois queremos o pior caso para garantir que tenhamos convergência em todas as outras;
* Variar os parâmetros: `activation`, `solver`, `hinde_layers_sizes`, `learning_rate_init` e `max_iter`;
* Nessa base que selecionamos, realizar um grid search e buscar as 5 melhores definições e realizar elas nas outras bases;
* Salvar as acurácias obtidas.

### 1.3.Parâmetros

* `activation`: Função de ativação utilizada. Iremos variar entre `identity`, `logistic`, `tanh` e `relu`;
* `solver`: Algoritmo usado para otimizar os pesos durante o treinamento. Iremos variar entre `sgd` e `adam`;
* `hidden_layer_sizes`: Parâmetro que define a quantidade de camadas encondidas e a quantidade de neurônios que cada camada terá. Para essa atividade, iremos utilizar de acordo com o número de classes (2) e features do maior dataset.
* `learning_rate_init`: Define a taxa de aprendizado inicial na MLP, controlando o tamanho dos passos dados pelo otimizador na atualização dos pesos. vamos variar entre  0.001, 0.01 e 0.1;
* `max_iter`: Número máximo de iterações. Iremos variar entre 500, 1000, 1500 e 2000.

## **2.Importando Bibliotecas**

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

## **3.Importando os Datasets**

Lembrando que temos 12 datasets, 6 deles foram os que perfomaram melhor na primeira atividade utilizando KNN, e o restante são o PCA para essas bases. Lembrando, também, que a primeira coluna é a label (0=gato, 1=cachorro).

In [2]:
hog_128_16 = pd.read_csv('../datasets/hog_128_16.csv')
hog_128_20 = pd.read_csv('../datasets/hog_128_20.csv')
cnn_VGG16_AVG_128 = pd.read_csv('../datasets/cnn_VGG16_AVG_128.csv')
cnn_VGG19_AVG_128 = pd.read_csv('../datasets/cnn_VGG19_AVG_128.csv')
cnn_VGG16_MAX_128 = pd.read_csv('../datasets/cnn_VGG16_MAX_128.csv')
cnn_VGG19_MAX_128 = pd.read_csv('../datasets/cnn_VGG19_MAX_128.csv')
hog_128_16_PCA = pd.read_csv('../datasets/hog_128_16_PCA.csv')
hog_128_20_PCA = pd.read_csv('../datasets/hog_128_20_PCA.csv')
cnn_VGG16_AVG_128_PCA = pd.read_csv('../datasets/cnn_VGG16_AVG_128_PCA.csv')
cnn_VGG19_AVG_128_PCA= pd.read_csv('../datasets/cnn_VGG19_AVG_128_PCA.csv')
cnn_VGG16_MAX_128_PCA= pd.read_csv('../datasets/cnn_VGG16_MAX_128_PCA.csv')
cnn_VGG19_MAX_128_PCA = pd.read_csv('../datasets/cnn_VGG19_MAX_128_PCA.csv')

In [3]:
#Instanciando um dict com todos os datasets
dataframes = {
    'hog_128_16': hog_128_16,
    'hog_128_20': hog_128_20,
    'cnn_VGG16_AVG_128': cnn_VGG16_AVG_128,
    'cnn_VGG19_AVG_128': cnn_VGG19_AVG_128,
    'cnn_VGG16_MAX_128': cnn_VGG16_MAX_128,
    'cnn_VGG19_MAX_128': cnn_VGG19_MAX_128,
    'hog_128_16_PCA': hog_128_16_PCA,
    'hog_128_20_PCA': hog_128_20_PCA,
    'cnn_VGG16_AVG_128_PCA': cnn_VGG16_AVG_128_PCA,
    'cnn_VGG19_AVG_128_PCA': cnn_VGG19_AVG_128_PCA,
    'cnn_VGG16_MAX_128_PCA': cnn_VGG16_MAX_128_PCA,
    'cnn_VGG19_MAX_128_PCA': cnn_VGG19_MAX_128_PCA
}

## **4.Setando os Parâmetros**

In [4]:
#Encontrando o dataset com o maior número de colunas
largest_dataset = max(dataframes.items(), key=lambda item: item[1].shape[1])
largest_dataset_name, largest_dataset_data = largest_dataset
largest_dataset_columns = largest_dataset_data.shape[1]

largest_dataset_name, largest_dataset_columns

('hog_128_16', 1765)

A maior base é a `hog_128_16` com 1765 colunas. Portanto, o parâmetro `hidden_layer_sizes` será dados de duas formas: 
* $HiddenLayerSize = \frac{(NClasses + NColunas)}{2} = 884$
* $HiddenLayerSize = NClasses + NColunas = 1767$

E, para cada um desses números, vamos fazer para uma camada e para duas camadas.

In [5]:
#grid de parâmetros
param_grid = { 
  'activation': ['identity', 'logistic', 'tanh', 'relu'],
  'solver': ['sgd', 'adam'],
  'hidden_layer_sizes': [(1767,), (884,), (884, 883), (442, 442)],
  'learning_rate_init': [0.001, 0.01, 0.1],
  'max_iter': [5000]
}

## **5.GridSearch**

Faremos um GridSearch, utilizando um holdout simples, para que recuperemos as 5 melhores configurações, e podermos aplicarmos ela nas outras bases.

In [6]:
# Carregar a base
data = hog_128_16

# Separando a label
X = data.drop(columns=['label'])
y = data['label'] 

# holdout
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalizando os Dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Instanciando o MLP
mlp = MLPClassifier(random_state=42)

# Configurando o GridSearchCV
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, n_jobs=-1, cv=3, verbose=2, scoring='accuracy')

# Ajustar o modelo com os dados de treino
grid_search.fit(X_train, y_train)

# Obtendo as 5 melhores configurações
best_params = grid_search.best_params_
top_5 = grid_search.cv_results_['params']

# Exibindo as 5 melhores configurações
sorted_top_5 = sorted(top_5, key=lambda x: grid_search.cv_results_['mean_test_score'][top_5.index(x)], reverse=True)[:5]
print("As 5 melhores configurações de parâmetros são:")
for i, config in enumerate(sorted_top_5, 1):
    print(f"{i}: {config}, Score: {grid_search.cv_results_['mean_test_score'][top_5.index(config)]}")


Fitting 3 folds for each of 96 candidates, totalling 288 fits


c:\Users\david\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
4 fits failed out of a total of 288.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\david\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\david\anaconda3\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 752, in fit
 

As 5 melhores configurações de parâmetros são:
1: {'activation': 'identity', 'hidden_layer_sizes': (884, 883), 'learning_rate_init': 0.1, 'max_iter': 5000, 'solver': 'sgd'}, Score: nan
2: {'activation': 'identity', 'hidden_layer_sizes': (442, 442), 'learning_rate_init': 0.1, 'max_iter': 5000, 'solver': 'sgd'}, Score: nan
3: {'activation': 'relu', 'hidden_layer_sizes': (442, 442), 'learning_rate_init': 0.01, 'max_iter': 5000, 'solver': 'sgd'}, Score: 0.6720430107526881
4: {'activation': 'tanh', 'hidden_layer_sizes': (442, 442), 'learning_rate_init': 0.1, 'max_iter': 5000, 'solver': 'adam'}, Score: 0.6702508960573477
5: {'activation': 'relu', 'hidden_layer_sizes': (442, 442), 'learning_rate_init': 0.001, 'max_iter': 5000, 'solver': 'sgd'}, Score: 0.6702508960573477


Com isso, temos as 5 melhores configurações:

In [7]:
# Obtendo as 10 melhores configurações
best_params = grid_search.best_params_
top_10 = grid_search.cv_results_['params']

# Exibindo as 10 melhores configurações
sorted_top_10 = sorted(top_10, key=lambda x: grid_search.cv_results_['mean_test_score'][top_10.index(x)], reverse=True)[:10]
print("As 10 melhores configurações de parâmetros são:")
for i, config in enumerate(sorted_top_10, 1):
    print(f"{i}: {config}, Score: {grid_search.cv_results_['mean_test_score'][top_10.index(config)]}")

As 10 melhores configurações de parâmetros são:
1: {'activation': 'identity', 'hidden_layer_sizes': (884, 883), 'learning_rate_init': 0.1, 'max_iter': 5000, 'solver': 'sgd'}, Score: nan
2: {'activation': 'identity', 'hidden_layer_sizes': (442, 442), 'learning_rate_init': 0.1, 'max_iter': 5000, 'solver': 'sgd'}, Score: nan
3: {'activation': 'relu', 'hidden_layer_sizes': (442, 442), 'learning_rate_init': 0.01, 'max_iter': 5000, 'solver': 'sgd'}, Score: 0.6720430107526881
4: {'activation': 'tanh', 'hidden_layer_sizes': (442, 442), 'learning_rate_init': 0.1, 'max_iter': 5000, 'solver': 'adam'}, Score: 0.6702508960573477
5: {'activation': 'relu', 'hidden_layer_sizes': (442, 442), 'learning_rate_init': 0.001, 'max_iter': 5000, 'solver': 'sgd'}, Score: 0.6702508960573477
6: {'activation': 'logistic', 'hidden_layer_sizes': (1767,), 'learning_rate_init': 0.1, 'max_iter': 5000, 'solver': 'adam'}, Score: 0.6666666666666666
7: {'activation': 'relu', 'hidden_layer_sizes': (884, 883), 'learning_rate

In [8]:
best_models = [
    MLPClassifier(activation= 'relu', hidden_layer_sizes= (442, 442), learning_rate_init= 0.01, max_iter= 5000, solver= 'sgd'),
    MLPClassifier(activation= 'tanh', hidden_layer_sizes= (442, 442), learning_rate_init= 0.1, max_iter= 5000, solver= 'adam'),
    MLPClassifier(activation= 'relu', hidden_layer_sizes= (442, 442), learning_rate_init= 0.001, max_iter= 5000, solver= 'sgd'),
    MLPClassifier(activation= 'logistic', hidden_layer_sizes= (1767,), learning_rate_init= 0.1, max_iter= 5000, solver= 'adam'),
    MLPClassifier(activation= 'relu', hidden_layer_sizes= (884, 883), learning_rate_init= 0.001, max_iter= 5000, solver= 'sgd')
]

## **6.Instanciando o Dataframe**

In [9]:
# Criando o multi_index com base nos dataframes
multi_index = []
for name in dataframes.keys():
    multi_index.extend([(name, '70/30'), (name, '10-fold CV')])

# Criando o DataFrame para armazenar as acurácias
accuracy_df = pd.DataFrame(index=pd.MultiIndex.from_tuples(multi_index), 
                           columns=['M1', 'M2', 'M3', 'M4', 'M5'])

## **7.Loop dos Modelos**

In [10]:
# Loop pelos dataframes no dicionário
for i, (df_name, df) in enumerate(dataframes.items(), start=1):
    print(f"\nIniciando processamento do dataframe {i}/{len(dataframes)}: {df_name}")
    
    # Separando X (features) e y (target)
    print(f"[{df_name}] Separando X (features) e y (target)...")
    X = df.drop(columns='label')  # Coluna alvo agora é 'label'
    y = df['label']
    
    # Divisão 70/30
    print(f"[{df_name}] Dividindo em treino (70%) e teste (30%)...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Holdout 70/30
    print(f"[{df_name}] Realizando Holdout (70/30)...")
    holdout_scores = []
    for j, model in enumerate(best_models, start=1):
        print(f"[{df_name}] Treinando modelo {j}/{len(best_models)} no Holdout...")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        holdout_scores.append(accuracy)
    
    # K-Fold com k=10
    print(f"[{df_name}] Realizando validação cruzada (K-Fold com k=10)...")
    kfold_scores = []
    for j, model in enumerate(best_models, start=1):
        print(f"[{df_name}] Avaliando modelo {j}/{len(best_models)} no K-Fold...")
        cv_scores = cross_val_score(model, X, y, cv=10, scoring='accuracy')
        kfold_scores.append(cv_scores.mean())
    
    # Armazenando os resultados no dataframe com o formato MultiIndex
    print(f"[{df_name}] Armazenando resultados no dataframe...")
    for idx, model_name in enumerate(['M1', 'M2', 'M3', 'M4', 'M5']):
        accuracy_df.loc[(df_name, '70/30'), model_name] = holdout_scores[idx]
        accuracy_df.loc[(df_name, '10-fold CV'), model_name] = kfold_scores[idx]
    
    # Imprimindo o dataframe atualizado
    print(f"[{df_name}] Processamento concluído. Dataframe atualizado:")
    print(accuracy_df)


Iniciando processamento do dataframe 1/12: hog_128_16
[hog_128_16] Separando X (features) e y (target)...
[hog_128_16] Dividindo em treino (70%) e teste (30%)...
[hog_128_16] Realizando Holdout (70/30)...
[hog_128_16] Treinando modelo 1/5 no Holdout...
[hog_128_16] Treinando modelo 2/5 no Holdout...
[hog_128_16] Treinando modelo 3/5 no Holdout...
[hog_128_16] Treinando modelo 4/5 no Holdout...
[hog_128_16] Treinando modelo 5/5 no Holdout...
[hog_128_16] Realizando validação cruzada (K-Fold com k=10)...
[hog_128_16] Avaliando modelo 1/5 no K-Fold...
[hog_128_16] Avaliando modelo 2/5 no K-Fold...
[hog_128_16] Avaliando modelo 3/5 no K-Fold...
[hog_128_16] Avaliando modelo 4/5 no K-Fold...
[hog_128_16] Avaliando modelo 5/5 no K-Fold...
[hog_128_16] Armazenando resultados no dataframe...
[hog_128_16] Processamento concluído. Dataframe atualizado:
                                        M1        M2        M3        M4  \
hog_128_16            70/30       0.620833  0.516667  0.608333  0.51

In [11]:
accuracy_df

M1        M2        M3        M4  \
hog_128_16            70/30       0.620833  0.516667  0.608333  0.516667   
                      10-fold CV  0.622816  0.498734  0.621472       0.5   
hog_128_20            70/30         0.6625  0.483333    0.6625  0.516667   
                      10-fold CV  0.669146  0.501266   0.66663  0.501266   
cnn_VGG16_AVG_128     70/30       0.483333  0.483333  0.541667  0.483333   
                      10-fold CV  0.513766  0.501266  0.522563       0.5   
cnn_VGG19_AVG_128     70/30       0.483333  0.483333     0.525  0.516667   
                      10-fold CV  0.511266       0.5       0.5       0.5   
cnn_VGG16_MAX_128     70/30       0.483333  0.483333  0.545833  0.516667   
                      10-fold CV  0.512484       0.5  0.511313       0.5   
cnn_VGG19_MAX_128     70/30       0.483333  0.483333  0.483333  0.483333   
                      10-fold CV  0.511218       0.5    0.5125  0.498734   
hog_128_16_PCA        70/30       0.629167    0.6375    0.6375  0.591667   
                      10-fold CV  0.645237  0.575127  0.624019  0.628892   
hog_128_20_PCA        70/30       0.633333    0.5875  0.645833  0.641667   
                      10-fold CV  0.625158  0.577532  0.633877  0.630142   
cnn_VGG16_AVG_128_PCA 70/30       0.633333    0.6125  0.666667  0.658333   
                      10-fold CV  0.660111   0.59019  0.658956  0.622579   
cnn_VGG19_AVG_128_PCA 70/30       0.641667  0.579167    0.6375  0.558333   
                      10-fold CV  0.613892  0.602658  0.613877    0.5875   
cnn_VGG16_MAX_128_PCA 70/30       0.620833  0.520833    0.6125  0.583333   
                      10-fold CV  0.615332  0.620253  0.601519  0.607769   
cnn_VGG19_MAX_128_PCA 70/30            0.6      0.55       0.6    0.6375   
                      10-fold CV  0.586313  0.558703  0.563813  0.568845   

                                        M5  
hog_128_16            70/30         0.6125  
                      10-fold CV  0.631535  
hog_128_20            70/30         0.6625  
                      10-fold CV   0.66288  
cnn_VGG16_AVG_128     70/30       0.533333  
                      10-fold CV   0.53125  
cnn_VGG19_AVG_128     70/30       0.483333  
                      10-fold CV  0.501266  
cnn_VGG16_MAX_128     70/30       0.483333  
                      10-fold CV  0.535079  
cnn_VGG19_MAX_128     70/30       0.583333  
                      10-fold CV  0.538766  
hog_128_16_PCA        70/30       0.645833  
                      10-fold CV  0.632706  
hog_128_20_PCA        70/30           0.65  
                      10-fold CV  0.627674  
cnn_VGG16_AVG_128_PCA 70/30       0.654167  
                      10-fold CV  0.661487  
cnn_VGG19_AVG_128_PCA 70/30       0.629167  
                      10-fold CV  0.617674  
cnn_VGG16_MAX_128_PCA 70/30       0.608333  
                      10-fold CV  0.625332  
cnn_VGG19_MAX_128_PCA 70/30       0.641667  
                      10-fold CV  0.597658

In [12]:
accuracy_df.to_excel("acurácias_mlp.xlsx")